# Functions


- Default Arguments
    - Mutable Default Arguments
        - Testing for ``False``
        - Testing for ``None``
    - Immutable Default Arguments
        - Coupled Calls
        - Decoupled Calls
        - Example: File Compression
- Positional Parameters
    - Example: Average
    - General Case
- Keyword Parameters
    - Example: Type Checking
    - General Case
- The Omnisignature
- Exploding Arguments
    - Exploding Positional Arguments
        - Star Unpacking
    - Exploding Keyword Arguments
        - Double-Star Unpacking
- Perfect Forwarding
- Keyword-Only Parameters
    - Forcing Keyword Arguments
- Positional-Only Parameters
    - Forcing Positional Arguments
- Annotations
    - ``mypy``
- Functions as Objects
    - As Arguments
    - As Return Values
    - Function Attributes
        - State
    - Function Internals
        - Default Arguments
        - Annotations
        - Bytecode
        - Constants
        - Local Variables
        - Arguments
        - Global Variables
        - Closures
        - Other Attributes
        - ArgSpec
- Decorators
    - 1st Order Decorators
    - Perfect Forwarding
    - Example: Tracing
    - Example: Caching
    - Wrapping Problems
    - ``functools.wraps``
    - 2nd Order Decorators
        - Implementing ``functools.wraps``
        - Example: Parametrized Tracing
    - Semi-Parametrized Decorators
    - Decorator Classes
        - 1st Order Decorator Classes
        - 2nd Order Decorator Classes
    - Class Decorators
        - Example: Thread-Safe Classes
- Generators
    - Iteration
        - Exhaustion
        - ``iter``
    - Indexing Generators
    - The Power of Generators
        - Infinite Generators
    - Nested Generators
    - Generator Comprehensions
    - Co-routines

## Default Arguments

### Mutable Default Arguments

In [1]:
def append(item, items=[]):
    items.append(item)
    return items

In [2]:
items = [1, 2, 3]
append(4, items)

[1, 2, 3, 4]

In [3]:
append(1)

[1]

In [4]:
append(1)

[1, 1]

#### Testing for ``False``

In [5]:
def append(item, items=None):
    if not items:
        items = []
    # items = items or []
    items.append(item)
    return items

In [6]:
items = [1, 2, 3]
append(4, items)

[1, 2, 3, 4]

In [7]:
append(1)

[1]

In [8]:
append(1)

[1]

In [9]:
items = []
append(1, items)
items

[]

#### Testing for ``None``

In [10]:
def append(item, items=None):
    if items is None:
        items = []
    # items = items if items is not None else []
    items.append(item)
    return items

In [11]:
items = [1, 2, 3]
append(4, items)

[1, 2, 3, 4]

In [12]:
append(1)

[1]

In [13]:
append(1)

[1]

In [14]:
items = []
append(1, items)
items

[1]

### Immutable Default Arguments

In [15]:
def greet(name='stranger'):
    print(f'Hello, {name}!')

In [16]:
greet('Dan')

Hello, Dan!


In [17]:
greet()

Hello, stranger!


#### Coupled Calls

In [18]:
log = []
def on_user(name='stranger'):
    greet(name)
    log.append(name)

In [19]:
on_user('Dan')

Hello, Dan!


In [20]:
on_user('Dan')

Hello, Dan!


In [21]:
on_user()

Hello, stranger!


In [22]:
def greet(name='you'):
    print(f'Hello, {name}!')

In [23]:
on_user()

Hello, stranger!


#### Decoupled Calls

In [24]:
def greet(name=None):
    if name is None:
        name = 'stranger'
    print(f'Hello, {name}')

In [25]:
log = []
def on_user(name=None):
    greet(name)
    log.append(name)

In [26]:
default_name = 'stranger'
def greet(name=None):
    if name is None:
        name = default_name
    print(f'Hello, {name}!')

In [27]:
on_user()

Hello, stranger!


In [28]:
default_name = 'you'
on_user()

Hello, you!


#### Example: File Compression

In [29]:
import gzip

def write(path, data, compress=True):
    fp = gzip.GzipFile(path, 'w') if compress else open(path, 'w')
    with fp:
        fp.write(data)

In [30]:
import json

def write_json(path, obj, compress=True):
    data = json.dumps(obj)
    write(path, data, compress)

## Positional Parameters

### Example: Average

In [31]:
def average_2(x, y):
    return (x + y) / 2

In [32]:
average_2(1, 2)

1.5

In [33]:
def average_3(x, y, z):
    return (x + y + z) / 3

In [34]:
average_3(1, 2, 3)

2.0

In [35]:
def average_n(xs):
    return sum(xs) / len(xs)

In [36]:
average_n([1, 2])

1.5

In [37]:
average_n([1, 2, 3])

2.0

In [38]:
def average(*xs):
    return sum(xs) / len(xs)

In [39]:
average(1, 2)

1.5

In [40]:
average(1, 2, 3)

2.0

### General Case

In [41]:
def f(x, y, *args):
    print(args)

In [42]:
f(1, 2)

()


In [43]:
f(1, 2, 3)

(3,)


In [44]:
f(1, 2, 3, 4)

(3, 4)


## Keyword Parameters

### Example: Type Checking

In [45]:
def validate_types(values, types):
    for key, type_ in types.items():
        if key not in values:
            continue
        value = values[key]
        if not isinstance(value, type_):
            raise TypeError(f'{key!r} must be {type_.__name__} (got {value.__class__.__name__})')

In [46]:
values = {'x': 1, 'y': 'Hello, world!'}

In [47]:
validate_types(values, {'x': int, 'y': str})

In [48]:
validate_types(values, {'x': int, 'y': int})

TypeError: 'y' must be int (got str)

In [49]:
def validate_types(values, **types):
    for key, type_ in types.items():
        if key not in values:
            continue
        value = values[key]
        if not isinstance(value, type_):
            raise TypeError(f'{key!r} must be {type_.__name__} (got {value.__class__.__name__})')

In [50]:
validate_types(values, x=int, y=str)

In [51]:
validate_types(values, x=int, y=int)

TypeError: 'y' must be int (got str)

### General Case

In [52]:
def f(x, y, **kwargs):
    print(kwargs)

In [53]:
f(x=1, y=2)

{}


In [54]:
f(x=1, y=2, z=3)

{'z': 3}


In [55]:
f(x=1, y=2, z=3, w=4)

{'z': 3, 'w': 4}


## The Omnisignature

In [56]:
def f(*args, **kwargs):
    print(args, kwargs)

In [57]:
f()

() {}


In [58]:
f(1, 2)

(1, 2) {}


In [59]:
f(x=1, y=2)

() {'x': 1, 'y': 2}


In [60]:
f(1, 2, x=1, y=2)

(1, 2) {'x': 1, 'y': 2}


## Exploding Arguments

### Exploding Positional Arguments

In [61]:
def average(*xs):
    return sum(xs) / len(xs)

In [62]:
numbers = [1, 2, 3]

In [63]:
average(numbers[0], numbers[1], numbers[2])

2.0

In [64]:
average(*numbers)

2.0

In [65]:
average(*[1, 2], 3, *[], 4, 5, *[6, 7, 8])

4.5

#### Star Unpacking

In [66]:
x = [*[1, 2], 3, *[], 4, 5, *[6, 7, 8]]
x

[1, 2, 3, 4, 5, 6, 7, 8]

### Exploding Keyword Arguments

In [67]:
def validate_types(values, **types):
    for key, type_ in types.items():
        if key not in values:
            continue
        value = values[key]
        if not isinstance(value, type_):
            raise TypeError(f'{key!r} must be {type_.__name__} (got {value.__class__.__name__})')

In [68]:
types = {'x': int, 'y': str}

In [69]:
validate_types(values, x=types['x'], y=types['y'])

In [70]:
validate_types(values, **types)

In [71]:
validate_types(values, **{'x': int}, y=str, **{'z': bool}, **{})

#### Double Star Unpacking

In [72]:
d = dict(**{'x': int}, y=str, **{'z': bool}, **{})
d

{'x': int, 'y': str, 'z': bool}

In [73]:
d1 = {'x': 1}
d2 = {'y': 2}
d3 = {**d1, **d2}
d3

{'x': 1, 'y': 2}

## Perfect Forwarding

In [74]:
def apply(f, *args, **kwargs):
    return f(*args, **kwargs)

In [75]:
def add(x, y):
    return x + y

In [76]:
apply(add, 1, 2)

3

## Keyword-Only Parameters

In [77]:
import sys

def print_(*args, sep=' ', end='\n'):
    sys.stdout.write(sep.join(args) + end)
    sys.stdout.flush()

In [78]:
print('Hello', 'world')

Hello world


In [79]:
print('Hello', 'world', sep=', ', end='!\n')

Hello, world!


### Forcing Keyword Arguments

In [80]:
import socket

def listen(port, backlog=1, reuseaddr=False):
    listener = socket.socket()
    listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listener.bind(('', port))
    listener.listen(backlog)
    return listener

```python
listen(8000, 5, True)
```

In [81]:
import socket

def listen(port, *, backlog=1, reuseaddr=False):
    listener = socket.socket()
    listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listener.bind(('', port))
    listener.listen(backlog)
    return listener

In [82]:
listen(8000, 5, True)

TypeError: listen() takes 1 positional argument but 3 were given

```python
listen(8000, backlog=5, reuseaddr=True)
```

## Positional-Only Parameters

```python
def add(x, y, /, z):
    return x + y + z
   
# OK:
add(1, 2, 3)

# OK:
add(1, 2, z=3)

# Not OK:
add(x=1, y=2, z=3)
```

### Forcing Positional Arguments

```python
def add(x, y, /):
    return x + y

# OK:
add(1, 2)

# Not OK:
add(x=1, y=2)
```

## Annotations

In [83]:
def add(x: int, y: int) -> int:
    return x + y

In [84]:
add(1, 2)

3

In [85]:
add('foo', 'bar')

'foobar'

### ``mypy``

In [86]:
%%bash

cat > /tmp/a.py <<EOF
def add(x: int, y: int) -> int:
    return x + y

add('foo', 'bar')
EOF

python -m mypy /tmp/a.py

/tmp/a.py:4: error: Argument 1 to "add" has incompatible type "str"; expected "int"
/tmp/a.py:4: error: Argument 2 to "add" has incompatible type "str"; expected "int"


CalledProcessError: Command 'b"\ncat > /tmp/a.py <<EOF\ndef add(x: int, y: int) -> int:\n    return x + y\n\nadd('foo', 'bar')\nEOF\n\npython -m mypy /tmp/a.py\n"' returned non-zero exit status 1.

In [87]:
import typing

def average(*args: typing.List[int]) -> int:
    return sum(args) / len(args)

T = typing.TypeVar('T')
def head(items: typing.Sequence[T]) -> T:
    return items[0]

## Functions as Objects

### As Arguments

In [88]:
def run_twice(f):
    f()
    f()

In [89]:
def hello():
    print('Hello, world!')

In [90]:
run_twice(hello)

Hello, world!
Hello, world!


### As Return Values

In [91]:
def create_power(n):
    def power(x):
        return x ** n
    return power

In [92]:
square = create_power(2)
square(2)

4

In [93]:
cube = create_power(3)
cube(2)

8

### Function Attributes

In [94]:
def add(x, y):
    'returns the sum of x and y'
    return x + y

In [95]:
add.__name__

'add'

In [96]:
add.__doc__

'returns the sum of x and y'

In [97]:
add.x = 1
add.x

1

#### State

In [98]:
def hello():
    hello.runs += 1
    print('Hello, world!')
hello.runs = 0

In [99]:
hello()

Hello, world!


In [100]:
hello()

Hello, world!


In [101]:
hello.runs

2

In [102]:
def greet(name=None):
    if name is None:
        name = greet.default_name
    print(f'Hello, {name}!')
greet.default_name = 'stranger'

In [103]:
greet()

Hello, stranger!


In [104]:
greet.default_name = 'you'
greet()

Hello, you!


### Function Internals

#### Default Arguments

In [105]:
def f(x=1):
    return x

In [106]:
f.__defaults__

(1,)

In [107]:
f.__defaults__ = (2,)

In [108]:
f()

2

#### Annotations

In [109]:
def f(x: int, y: str) -> bool:
    pass

In [110]:
f.__annotations__

{'x': int, 'y': str, 'return': bool}

#### Bytecode

In [111]:
def f():
    return 1 / 0

In [112]:
f()

ZeroDivisionError: division by zero

In [113]:
def f():
    return 'Hello, world!

SyntaxError: EOL while scanning string literal (<ipython-input-113-e0594def10a6>, line 2)

#### Constants

In [114]:
def f():
    return 42

In [115]:
import dis

dis.disassemble(f.__code__)

  2           0 LOAD_CONST               1 (42)
              2 RETURN_VALUE


In [116]:
f.__code__.co_consts

(None, 42)

#### Local Variables

In [117]:
def f():
    x = 6
    y = 7
    z = x * y
    return z

In [118]:
dis.disassemble(f.__code__)

  2           0 LOAD_CONST               1 (6)
              2 STORE_FAST               0 (x)

  3           4 LOAD_CONST               2 (7)
              6 STORE_FAST               1 (y)

  4           8 LOAD_FAST                0 (x)
             10 LOAD_FAST                1 (y)
             12 BINARY_MULTIPLY
             14 STORE_FAST               2 (z)

  5          16 LOAD_FAST                2 (z)
             18 RETURN_VALUE


In [119]:
f.__code__.co_varnames

('x', 'y', 'z')

In [120]:
f.__code__.co_consts

(None, 6, 7)

#### Arguments

In [121]:
def f(x, y):
    z = x * y
    return z

In [122]:
dis.disassemble(f.__code__)

  2           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_MULTIPLY
              6 STORE_FAST               2 (z)

  3           8 LOAD_FAST                2 (z)
             10 RETURN_VALUE


In [123]:
f.__code__.co_varnames

('x', 'y', 'z')

In [124]:
f.__code__.co_consts

(None,)

In [125]:
f.__code__.co_argcount

2

In [126]:
f.__code__.co_varnames[:f.__code__.co_argcount]

('x', 'y')

#### Global Variables

In [127]:
x = 1
def f():
    return x

In [128]:
dis.disassemble(f.__code__)

  3           0 LOAD_GLOBAL              0 (x)
              2 RETURN_VALUE


In [129]:
f.__code__.co_varnames

()

In [130]:
f.__code__.co_names

('x',)

In [131]:
f.__globals__['x']

1

In [132]:
import types
f2 = types.FunctionType(f.__code__, {'x': 2})

In [133]:
f2.__globals__['x']

2

In [134]:
f2()

2

#### Closures

In [135]:
def f():
    x = 1
    def g():
        return x
    return g

g = f()

In [136]:
dis.disassemble(g.__code__)

  4           0 LOAD_DEREF               0 (x)
              2 RETURN_VALUE


In [137]:
g.__closure__

(<cell at 0x111928c18: int object at 0x100984a00>,)

In [138]:
g.__closure__[0].cell_contents

1

In [139]:
# No easy way to create cell objects...
def get_cell(n):
    def _():
        return n
    return _.__closure__[0]

In [140]:
g2 = types.FunctionType(
    g.__code__,
    g.__globals__,
    g.__name__,
    g.__defaults__,
    (get_cell(2),)
)

In [141]:
g2()

2

#### Other Attributes

In [142]:
%%bash

cat > /tmp/b.py <<EOF
def f():
    pass

def g():
    pass
EOF

In [143]:
import sys
sys.path.append('/tmp')

import b

In [144]:
b.f.__code__.co_filename

'/tmp/b.py'

In [145]:
b.f.__code__.co_firstlineno

1

In [146]:
b.g.__code__.co_firstlineno

4

### ArgSpec

In [147]:
def f(x, y=2, *args, z=3, **kwargs) -> 'Hello, world!':
    pass

In [148]:
inspect.getfullargspec(f)

NameError: name 'inspect' is not defined

## Decorators

### 1st Order Decorators

In [149]:
def double(f):
    def wrapper(x):
        return f(x) * 2
    return wrapper

In [150]:
def inc(x):
    return x + 1

In [151]:
inc = double(inc)

In [152]:
inc(1) # (1 + 1) * 2 = 4

4

In [153]:
@double
def inc(x):
    return x + 1

In [154]:
inc(1)

4

### Perfect Forwarding

In [155]:
def double(f):
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs) * 2
    return wrapper

In [156]:
@double
def inc(x):
    return x + 1

In [157]:
inc(1)

4

In [158]:
@double
def add(x, y):
    return x + y

In [159]:
add(1, 2) # (1 + 2) * 2 = 6

6

### Example: Tracing

In [160]:
def trace(f):
    def wrapper(*args, **kwargs):
        print(f'enter {f.__name__}')
        try:
            return f(*args, **kwargs)
        finally:
            print(f'leave {f.__name__}')
    return wrapper

In [161]:
@trace
def div(x, y):
    return x / y

In [162]:
div(4, 2)

enter div
leave div


2.0

In [163]:
div(1, 0)

enter div
leave div


ZeroDivisionError: division by zero

In [164]:
import inspect


def trace(f):
    def wrapper(*args, **kwargs):
        call = ''
        if args:
            call += ', '.join(repr(arg) for arg in args)
        if kwargs:
            call += ', '.join(f'{key}={value!r}' for key, value in kwargs.items())
        print(f'enter {f.__name__}({call})')
        try:
            result = f(*args, **kwargs)
            print(f'leave {f.__name__}({call}): {result!r}')
            return result
        except Exception as error:
            print(f'leave {f.__name__}({call}) on error: {error}')
            raise
    return wrapper

In [165]:
@trace
def div(x, y):
    return x / y

In [166]:
div(4, 2)

enter div(4, 2)
leave div(4, 2): 2.0


2.0

In [167]:
div(1, 0)

enter div(1, 0)
leave div(1, 0) on error: division by zero


ZeroDivisionError: division by zero

### Example: Caching

In [168]:
def fib(n):
    return n if n <= 1 else fib(n-1) + fib(n-2)

In [169]:
%time fib(35)

CPU times: user 4.65 s, sys: 7.53 ms, total: 4.66 s
Wall time: 4.66 s


9227465

In [170]:
def cache(f):
    cache = {}
    def wrapper(*args, **kwargs):
        token = args + tuple(kwargs.items())
        if token not in cache:
            cache[token] = f(*args, **kwargs)
        return cache[token]
    return wrapper

In [171]:
@cache
def fib(n):
    return n if n <= 1 else fib(n-1) + fib(n-2)

In [172]:
%time fib(35)

CPU times: user 111 µs, sys: 25 µs, total: 136 µs
Wall time: 141 µs


9227465

### Wrapping Problems

In [173]:
def double(f):
    def wrapper(*args, **kwargs):
        return f(*arg, **kwargs) * 2
    return wrapper

In [174]:
@double
def add(x, y):
    'returns the sum of x and y'
    return x + y

In [175]:
add.__name__

'wrapper'

In [176]:
add.__doc__

### `functools.wraps`

In [177]:
import functools

In [178]:
def double(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs) * 2
    return wrapper

In [179]:
@double
def add(x, y):
    'returns the sum of x and y'
    return x + y

In [180]:
add.__name__

'add'

In [181]:
add.__doc__

'returns the sum of x and y'

In [182]:
add(1, 2)

6

### 2nd Order Decorators

In [183]:
def multiply(m):
    def decorator(f):
        def wrapper(*args, **kwargs):
            return f(*args, **kwargs) * m
        return wrapper
    return decorator

In [184]:
double = multiply(2)

In [185]:
@double
def inc(x):
    return x + 1

inc(1)

4

In [186]:
triple = multiply(3)

In [187]:
@triple
def inc(x):
    return x + 1

inc(1)

6

In [188]:
@multiply(2)
def inc(x):
    return x + 1

inc(1)

4

In [189]:
@multiply(3)
def inc(x):
    return x + 1

inc(1)

6

#### Implementing ``functools.wraps``

In [190]:
def wraps(f):
    def decorator(d):
        d.__name__ = f.__name__
        d.__doc__ = f.__doc__
        return d
    return decorator

In [191]:
def double(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs) * 2
    return wrapper

In [192]:
@double
def add(x, y):
    'returns the sum of x and y'
    return x + y

In [193]:
add.__name__

'add'

In [194]:
add.__doc__

'returns the sum of x and y'

In [195]:
add(1, 2)

6

#### Exmaple: Parametrized Tracing

In [196]:
def trace(log):
    def decorator(f):
        def wrapper(*args, **kwargs):
            log(f'enter {f.__name__}')
            try:
                return f(*args, **kwargs)
            finally:
                log(f'leave {f.__name__}')
        return wrapper
    return decorator

In [197]:
@trace(print)
def inc(x):
    return x + 1

inc(1)

enter inc
leave inc


2

In [198]:
fp = open('/tmp/log', 'w')

def write(line):
    fp.write(line + '\n')
    fp.flush()

@trace(write)
def inc(x):
    return x + 1

inc(x)

!cat /tmp/log

enter inc
leave inc


### Semi-Parametrized Decorators

In [199]:
def trace(f=None, *, log=None):
    if f is None:
        return lambda f: trace(f, log=log)
    if log is None:
        log = print
    def wrapper(*args, **kwargs):
        log(f'enter {f.__name__}')
        try:
            return f(*args, **kwargs)
        finally:
            log(f'leave {f.__name__}')
    return wrapper

In [200]:
@trace
def inc(x):
    return x + 1

inc(1)

enter inc
leave inc


2

In [201]:
fp = open('/tmp/log', 'w')

def write(line):
    fp.write(line + '\n')
    fp.flush()

@trace(log=write)
def inc(x):
    return x + 1

inc(x)

!cat /tmp/log

enter inc
leave inc


### Decorator Classes

#### 1st Order Decorator Classes

In [202]:
class double:
    
    def __init__(self, f):
        self._f = f
    
    def __call__(self, *args, **kwargs):
        return self._f(*args, **kwargs)

In [203]:
@double
def inc(x):
    return x + 1

inc(1)

2

In [204]:
class cache:
    
    def __init__(self, f):
        self._f = f
        self._cache = {}
    
    def __call__(self, *args, **kwargs):
        token = args + tuple(kwargs.items())
        if token not in self._cache:
            self._cache[token] = self._f(*args, **kwargs)
        return self._cache[token]
    
    def clear_cache(self):
        self._cache.clear()

In [205]:
import time

@cache
def f(x):
    time.sleep(1)
    return x

In [206]:
%time f(1)

CPU times: user 622 µs, sys: 1.09 ms, total: 1.72 ms
Wall time: 1.01 s


1

In [207]:
%time f(1)

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 11.9 µs


1

In [208]:
f.clear_cache()

In [209]:
%time f(1)

CPU times: user 836 µs, sys: 1.12 ms, total: 1.96 ms
Wall time: 1 s


1

#### 2nd Order Decorator Classes

In [210]:
class multiply:
    
    def __init__(self, m):
        self.m = m
    
    def __call__(self, f):
        def wrapper(*args, **kwargs):
            return f(*args, **kwargs) * self.m
        return wrapper

In [211]:
@multiply(2)
def inc(x):
    return x + 1

inc(1)

4

In [212]:
@multiply(3)
def inc(x):
    return x + 1

inc(1)

6

### Class Decorators

In [213]:
def double(f):
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs) * 2
    return wrapper

def double_all(cls):
    for k, v in cls.__dict__.items():
        if not callable(v):
            continue
        setattr(cls, k, double(v))
    return cls

In [214]:
@double_all
class A:
    
    def inc(self, x):
        return x + 1
    
    def add(self, x, y):
        return x + y

In [215]:
a = A()

In [216]:
a.inc(1)

4

In [217]:
a.add(1, 2)

6

#### Example: Thread-Safe Classes

In [218]:
import threading

def threadsafe(cls):
    lock = threading.Lock()
    def synchronized(f):
        def wrapper(*args, **kwargs):
            with lock:
                return f(*args, **kwargs)
        return wrapper
    for k, v in cls.__dict__.items():
        if not callable(v):
            continue
        setattr(cls, k, synchronized(v))
    return cls

In [219]:
@threadsafe
class File:
    
    def __init__(self, path):
        self.path = path
    
    def write(self, data):
        with open(self.path, 'w') as writer:
            writer.write(data)
    
    def read(self):
        with open(self.path, 'r') as reader:
            return reader.read()

## Generators

In [220]:
def gen():
    yield 1
    yield 2
    yield 3

In [221]:
g = gen()
g

<generator object gen at 0x111e5bf68>

In [222]:
next(g)

1

In [223]:
next(g)

2

In [224]:
next(g)

3

In [225]:
next(g)

StopIteration: 

### Iteration

In [226]:
for n in gen():
    print(n)

1
2
3


#### Exhaustion

In [227]:
g = gen()
for n in g:
    print(n)

1
2
3


In [228]:
for n in g:
    print(n)

#### ``iter``

In [229]:
x = [1, 2, 3]

In [230]:
i = iter(x)
i

In [231]:
next(i)

1

In [232]:
next(i)

2

In [233]:
next(i)

3

In [234]:
next(i)

StopIteration: 

### Indexing Generators

In [235]:
g = gen()
g[0]

TypeError: 'generator' object is not subscriptable

In [236]:
x = [1, 2, 3]

In [237]:
reversed(x)[0]

TypeError: 'list_reverseiterator' object is not subscriptable

In [238]:
list(reversed(x))[0]

3

In [239]:
next(reversed(x))

3

In [240]:
x = []

In [241]:
next(reversed(x))

StopIteration: 

In [242]:
next(reversed(x), 'empty')

'empty'

### The Power of Generators

In [243]:
import time

def gen():
    time.sleep(1)
    yield 1
    time.sleep(1)
    yield 2
    time.sleep(1)
    yield 3

In [244]:
%%time

if 2 in gen():
    print('has 2')

has 2
CPU times: user 776 µs, sys: 1.13 ms, total: 1.91 ms
Wall time: 2.01 s


In [245]:
%%time

if 2 in list(gen()):
    print('has 2')

has 2
CPU times: user 997 µs, sys: 1.46 ms, total: 2.46 ms
Wall time: 3.01 s


In [246]:
import os

w = os.walk('/')
w

<generator object walk at 0x111e61b48>

#### Infinite Generators

In [247]:
def infinite():
    n = 0
    while True:
        yield n
        n += 1

In [248]:
def connections(port):
    listener = socket.socket()
    listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    listener.bind(('', port))
    listener.listen(5)
    while True:
        connection, address = listener.accept()
        yield connection

```python
for connection in listen(8000):
    handle(connection)
```

### Nested Generators

In [249]:
def gen1():
    yield 1
    yield 2

In [250]:
def gen2():
    yield 3

In [251]:
def gen():
    yield from gen1()
    yield from gen2()

In [252]:
for n in gen():
    print(n)

1
2
3


### Generator Comprehensions

In [253]:
g = (i**2 for i in range(5))
g

<generator object <genexpr> at 0x111e61db0>

In [254]:
for n in g:
    print(n)

0
1
4
9
16


In [255]:
sum(i**2 for i in range(5))

30

In [256]:
sum([i**2 for i in range(5)])

30

In [257]:
next(i for i in range(1000) if i**2 > 1000)

32

### Coroutines

In [258]:
def gen():
    try:
        x = yield 1
        print(f'x = {x}')
        y = yield 2
        print(f'y = {y}')
        z = yield 3
        print(f'z = {z}')
    except Exception:
        print('error')

In [259]:
g = gen()

In [260]:
next(g)

1

In [261]:
next(g)

x = None


2

In [262]:
g.send('Hello, world!')

y = Hello, world!


3

In [263]:
g.throw(ValueError())

error


StopIteration: 